In [20]:

import requests
from bs4 import BeautifulSoup

import csv
import pandas as pd
import re
import math
url = "https://www.kabum.com.br/hardware/placa-de-video-vga"
base_url = "https://www.kabum.com.br/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
site = requests.get(url, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')
qtd_itens = soup.find('div', id='listingCount').get_text().strip()
index = qtd_itens.find(' ')
qtd = qtd_itens[:index]
ultima_pagina = math.ceil(int(qtd)/20)
lista_produtos = []




for i in range(1, ultima_pagina+1):
    url_pag = f'https://www.kabum.com.br/hardware/placa-de-video-vga?page_number={i}&page_size=20&facet_filters=&sort=most_searched'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')

    produtos = soup.find_all('div', class_='productCard')

    for produto in produtos:
         if produto.find('div', class_=re.compile('unavailablePricesCard')):
             continue

         link = base_url + produto.find('a', class_=re.compile('productLink')).get('href').strip()

         # Faz a requisição para a página do produto
         site_produto = requests.get(link, headers=headers)
         soup_produto = BeautifulSoup(site_produto.content, 'html.parser')
         infor = soup_produto.find('div', class_=re.compile('col-purchase'))
         #encontrar preços no pix
         preco_pix = None
         if infor:
             preco_pix_element = infor.find('h4', class_='finalPrice')
             if preco_pix_element:
                 preco_pix = preco_pix_element.get_text().strip()
         #encontrar nome   
         produto = None
         if infor:
             produto_element = infor.find('h1', class_='cMCMNo')
             if produto_element:
                 produto = produto_element.get_text().strip()

    
         #encontrar preços original
         preco_original = None
         if infor:
             preco_original_element = infor.find('span', class_='oldPrice')
             if preco_original_element:
                 preco_original = preco_original_element.get_text().strip()
          #encontrar preços no prazo
         regular_price_tag = None
         if infor:
             regular_price_element = infor.find('b', class_='regularPrice')
             if regular_price_element:
                 regular_price_tag = regular_price_element.get_text().strip()

         dados_produto = {
             'produtos': produto,
             'preço pix': preco_pix,
             'preco a prazo': regular_price_tag,
             'preço original': preco_original,
             'link': link,
         }

         lista_produtos.append(dados_produto)

         #print(produto, preco_pix, preco_original, link)

In [21]:
#csv
with open('placadeVideo.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['produtos', 'preço pix', 'preco a prazo', 'preço original', 'link'])
    writer.writeheader()  # Escreve o cabeçalho no arquivo

    writer.writerows(lista_produtos)

In [22]:
df = pd.DataFrame(lista_produtos)

In [23]:
df

,produtos,preço pix,preco a prazo,preço original,link
0,Placa de Vídeo RTX 3060 Asus Dual O12G V2 NVID...,"R$ 2.199,99","R$ 2.444,43","R$ 2.743,32",https://www.kabum.com.br//produto/164854/placa...
1,Placa de Vídeo RX 6600 CLD 8G ASRock AMD Radeo...,"R$ 1.449,99","R$ 1.705,87","R$ 2.223,52",https://www.kabum.com.br//produto/235984/placa...
2,Placa de Vídeo RTX 3060 Ti 1-Click OC Galax NV...,"R$ 2.399,99","R$ 2.666,66","R$ 3.144,43",https://www.kabum.com.br//produto/191789/placa...
3,Placa de Vídeo RX 6750 XT Mech 2X 12G OC MSI A...,"R$ 2.559,99","R$ 2.844,43","R$ 3.222,21",https://www.kabum.com.br//produto/417853/placa...
4,Placa de Vídeo RTX 3060 Twin Edge OC Zotac Gam...,"R$ 2.279,99","R$ 2.533,32","R$ 2.666,66",https://www.kabum.com.br//produto/148655/placa...
...,...,...,...,...,...
521,Placa De Video 4gb R7 240 Ddr5 128bits Vga/hdm...,"R$ 568,00","R$ 597,90",None,https://www.kabum.com.br//produto/328626/placa...
522,Bloco VGA Barrow Para Rtx 3080 (ti) E Rtx 3090...,"R$ 1.199,00",None,None,https://www.kabum.com.br//produto/332084/bloco...
523,Placa de Vídeo Asus NVIDIA GeForce Dual RTX305...,"R$ 15.075,01",None,None,https://www.kabum.com.br//produto/348710/placa...
524,Placa de Vídeo MSI NVIDIA GeForce Gaming X RTX...,"R$ 2.694,11","R$ 2.835,90",None,https://www.kabum.com.br//produto/367903/placa...
